Brainstorming:
- what is a context
- yield syntax
- `__enter__`, `__exit__` syntax
- when to + usecases (hushing logs, resource management, telemetry)
- Aside on advanced exception handling

Context managers
================

**Outline**:

## The notion of context

:hourglass: 20 min

Consider the following example:

In [6]:
import sys
import io

stdout_ = sys.stdout
stdout_buffer = io.StringIO()
try:
    sys.stdout = stdout_buffer

    # Code you actually want to run
    print("This is a call to the print function")
    
finally:
    sys.stdout = stdout_


In [7]:
stdout_buffer.getvalue()

'This is a call to the print function\n'

In this example, the `try`/`finally` construct defines a *context*. The goal of the context is to isolate some part of the execution. In this specific example, the isolation aims at capturing (notably) the `print` calls.

The first few statements of the function define the *setup* of the context (saving the default print buffer, overwriting the default print buffer). The whole of the `finally` block defines the *teardown* of the context (restoring the default print buffer).

Those kinds of contexts are often used in programs to make sure the teardown part will be executed properly, even when an exception occurs in the middle of the code.

Examples include:
- resource management: making sure a resource (file descriptor, lock/semaphore, etc.) are release properly;
- buffering: making sure a buffer is flushed;
- transaction: ensuring atomicity and consistency;
- monckeypatching: restoring the proper code (cf. example);
- parametrization: setting some configuration for the context (see eg. Airflow, PyTorch)

Although the notion of context is very useful, the above code has some drawbacks:
- the code of the context is mingled with the rest of the code, making it unclear which is which;
- the context is not re-usable for another function.

Python provides a very elegant mechanism to encapsulate the context code via the `with` keyword:

In [18]:
import contextlib
import io


print("Before context")
with contextlib.redirect_stdout(io.StringIO()) as buffer:
    print("This is a call to the print function")

print("After context")
print(f"Buffer: {buffer.getvalue()}")

Before context
After context
Buffer: This is a call to the print function



The `redirect_stdout` has a much more limited impact on the code, there is no more mingling of contextual and application code, and the code is now re-usable!

## Custom context managers

### The short syntax

:hourglass: 10 min

Python provides two syntaxes for creating context managers. The short version makes use of the `contexlib` decorator and is quite straightforward. You decorate a single-element generator:

In [19]:
from typing import Optional, Iterator
import io
import contextlib
import sys

@contextlib.contextmanager
def redirect_stdout(buffer: Optional[io.StringIO] = None) -> Iterator[io.StringIO]:
    if buffer is None:
        buffer = io.StringIO()
    
    stdout_ = sys.stdout
    try:
        sys.stdout = buffer
        yield buffer
    finally:
        sys.stdout = stdout_


print("Before context")
with redirect_stdout() as buffer:
    print("This is a call to the print function")

print("After context")
print(f"Buffer: {buffer.getvalue()}")

Before context
After context
Buffer: This is a call to the print function



### The powerful syntax

:hourglass: 10 min


While useful for simple tasks, you sometimes want the full flexibility of having an object serving as a context manager. This can be done via the `__enter__` and `__exit__` pair of methods.

In [20]:
from typing import Optional
from types import TracebackType
import io
import sys


class STDOutCatcher:
    def __init__(self, buffer: Optional[io.StringIO] = None) -> None:
        if buffer is None:
            buffer = io.StringIO()

        self._buffer = buffer
        self._sysstdout: Optional[io.StringIO] = None

    def __enter__(self) -> io.StringIO:
        self._sysstdout = sys.stdout
        sys.stdout = self._buffer
        return self._buffer
    
    def __exit__(
        self,
        exc_type: Optional[type[BaseException]],
        exc_val: Optional[BaseException],
        exc_tb: Optional[TracebackType],
    ) -> None:
        sys.stdout = self._sysstdout


print("Before context")
with STDOutCatcher() as buffer:
    print("This is a call to the print function")

print("After context")
print(f"Buffer: {buffer.getvalue()}")

Before context
After context
Buffer: This is a call to the print function



### Exercise

:hourglass: 30 min

Create a context manager to log the start/end of a portion of code, as well as the completion time. You can use the `from time import perf_counter` to track the duration. 



:coffee: 15 min

## The tryforce of error handling

:hourglass: 20 min

Can you predict what will be printed:

In [ ]:
def runtime_stop():
    raise RuntimeError()

try:
    print("the cat")
    runtime_stop()
    print("the dog")
except RuntimeError:
    print("the duck")
except ValueError:
    print("the lion")
except (RuntimeError, ValueError):
    print("the fox")
except Exception:
    print("the fish")
finally:
    print("the mouse")

What is the difference between the following snippets?

In [ ]:
try:
    runtime_stop()
except RuntimeError as e:
    raise e


In [ ]:
try:
    runtime_stop()
except RuntimeError as e:
    raise


In [ ]:
try:
    runtime_stop()
except RuntimeError as e:
    raise RuntimeError(2) from e

In [ ]:
import sys

try:
    runtime_stop()
except RuntimeError as e:
    tb = sys.exc_info()[2]
    raise RuntimeError(2).with_traceback(tb)

**always** provide the except clause with an exception class:

In [36]:
try:
    raise KeyboardInterrupt()
except:
    pass

In [ ]:
try:
    raise KeyboardInterrupt()
except Exception:
    pass

It is possible to create custom exceptions by inheriting from an exception class (see https://docs.python.org/3.9/library/exceptions.html#inheriting-from-built-in-exceptions).

Although this is encouraged in some languages, not so much in Python, where the exception hierarchy already provides comprehensive classes. In particular, it is worth knowing the following:
- **AssertionError**: raised when an assertion is violated (raised via the assert statement);
- **AttributeError**: raised when trying to access an attribute of an object which does not exist;
- **RuntimeError**: raised when something occurs at runtime and no more specific exception can be used;
- **NotImplementedError**: raised by an abstract method (*);
- **TypeError**: raised when the object is not of the proper type for some operation;
- **ValueError**: raised when the value is inappropriate (but the type is correct).

Some remarks:
- (*) the `NotImplementedError` exception must not be used to restrict what a class can do (cf. the substitution principle);
- Do not confuse `NotImplementedError` and `NotImplemented`;
- some exceptions serve as control flow (eg. `GeneratorExit`, `StopIteration`);
- :skull: Python 3.11 provides support for "exception groups" (https://peps.python.org/pep-0654/).

## Singleton pattern



## x. Closing words

**Dunderscore**:
- `__enter__`
- `__exit__`